<a href="https://colab.research.google.com/github/Avelez164/Data-Science-Big-Data/blob/main/Homework9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group Names:

*   Antonio Velez
*   Farren Tanudjaja

# 1. How does a Distributed File System protect against data loss due to a computer crash? Answer in 2-3 sentences.

By distributing data across multiple computers. Distributing data across multiple computers can improves data retrieval and storage performance.

# 2. Consider the problem of counting all length-3 sequences, including overlapping sequences, i.e., the number of occurrences of AGA, GAT, ATA, … using the following Map-Reduce program.

In [ ]:
%pip install mockr

In [ ]:
from mockr import run_stream_job
def map_fn(chunk):
  for i in range(len(chunk)-2):
    w = chunk[i:i+3]
    yield(w, 1)


def reduce_fn(key, values):
    yield (key, sum(values))

input_str = "AGATAGA"
results = run_stream_job(input_str, map_fn, reduce_fn)
print(results)

[('AGA', 2), ('GAT', 1), ('ATA', 1), ('TAG', 1)]


Below is a string containing a DNA sequence:

**AGATAGA**

Answer the following questions for this DNA sequence.

    Hint: you can run this code with mockr.


a) What are ALL the key, value pairs output by the Map step?

AGA, 1   AGA, 1   GAT, 1   ATA, 1   TAG, 1


b) What are ALL the key, value pairs input to the Reduce steps?

(AGA, 2)

(GAT, 1)

(ATA, 1)

(TAG, 1)

c) What is the output after the Reduce steps?

'AGA': 2

'GAT': 1

'ATA': 1

'TAG': 1

# 3. Consider the following Map-Reduce program to process a data frame.


In [ ]:
from mockr import run_stream_job
def map_fn(chunk):
  inputvalue = chunk.squeeze()
  if inputvalue.arr_delay > 30:
    yield(inputvalue.dest, 1)


def reduce_fn(key, values):
  yield(key, ())


If the input data frame is:


| | flight | carrier | arr_delay | dest |
|-|--------|---------------|---------------|---------|
| | UA | 345             | 20        | LAX      |
| | UA | 452        | 38         | MIA      |
| | DL | 428         | 4         | MIA      |
| | DL | 567            | 42         | ORD      |


Answer the following questions.

    Hint: you can run this code with mockr.

1) How many times will map() be executed?

Map will be executed 4 times


2) What are ALL the (key, value) pairs output by the Map functions?

mapper only yields when arr_delay>30.

Row0: arr_delay = 20 -> no output

Row1: arr_delay = 38 -> yields ("MIA", 1)

Row2: arr_delay = 4 -> no output

Row3: arr_delay = 42 -> yields ("ORD", 1)



3) What are ALL the (key, value) pairs input to the Reduce functions?

key "MIA" with values [1]

key "ORD" with values [1]


4) What is the output (keys only) output by the Reduce functions?

'MIA'
'ORD'

5) What does the above map-reduce program do?

SELECT DISTINCT dest

  FROM flights

 WHERE arr_delay > 30;



# 4. Use mockr to answer part 12 of question 1 from Homework 3  [code, results]:


    Load the R dataset flights from the nycflights13 package. The table contains flight arrival and departure data…


12. Show the average total delay for every departure-arrival city pair.

Do not alter the flights data frame; all data processing should take place in your map_fn() and reduce_fn().


In [ ]:
from mockr import run_pandas_job
import statsmodels.api as sm
import pandas as pd

flights = sm.datasets.get_rdataset('flights', 'nycflights13').data

def map_fn(chunk):
    chunk = chunk.dropna(subset=['dep_delay','arr_delay'])
    for _, row in chunk.iterrows():
        total = row.dep_delay + row.arr_delay
        yield ((row.origin, row.dest), total)

def reduce_fn(key, values):
    vals = list(values)
    yield (key, sum(vals)/len(vals))

results = run_pandas_job(flights, map_fn, reduce_fn, n_chunks=4)

df = pd.DataFrame([
    {'origin': o, 'dest': d, 'avg_total_delay': avg}
    for (o,d), avg in results
]).sort_values(['origin','dest']).reset_index(drop=True)

df


,origin,dest,avg_total_delay
0,EWR,ALB,37.844498
1,EWR,ANC,10.375000
2,EWR,ATL,28.626743
3,EWR,AUS,11.000000
4,EWR,AVL,17.382470
...,...,...,...
218,LGA,SYR,28.401487
219,LGA,TPA,19.394167
220,LGA,TVC,39.383562
221,LGA,TYS,16.600000


In [ ]:
def colab2pdf():
  # @title Download Notebook in PDF Format{display-mode:'form'}
  !apt-get install -yqq --no-install-recommends librsvg2-bin>/dev/null;
  import contextlib,datetime,google,io,IPython,ipywidgets,json,locale,nbformat,os,pathlib,requests,urllib,warnings,werkzeug,yaml,re;locale.setlocale(locale.LC_ALL,'en_US.UTF-8');warnings.filterwarnings('ignore',category=nbformat.validator.MissingIDFieldWarning);
  %matplotlib inline
  def convert(b):
    try:
      s.value='🔄 Converting';b.disabled=True
      n=pathlib.Path(werkzeug.utils.secure_filename(urllib.parse.unquote(requests.get(f'http://{os.environ["COLAB_JUPYTER_IP"]}:{os.environ["KMP_TARGET_PORT"]}/api/sessions').json()[0]['name'])))
      p=pathlib.Path('/content/pdfs')/f'{datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S")}_{n.stem}';p.mkdir(parents=True,exist_ok=True);nb=nbformat.reads(json.dumps(google.colab._message.blocking_request('get_ipynb',timeout_sec=600)['ipynb']),as_version=4)
      u=[u for c in nb.cells if c.get('cell_type')=='markdown' for u in re.findall(r'!\[.*?\]\((https?://.*?)\)',c['source']) if requests.head(u,timeout=5).status_code!=200]
      if u:raise Exception(f"Bad Image URLs: {','.join(u)}")
      nb.cells=[cell for cell in nb.cells if '--Colab2PDF' not in cell.source]
      nb=nbformat.v4.new_notebook(cells=nb.cells or [nbformat.v4.new_code_cell('#')]);nbformat.validator.normalize(nb)
      nbformat.write(nb,(p/f'{n.stem}.ipynb').open('w',encoding='utf-8'))
      with (p/'config.yml').open('w', encoding='utf-8') as f: yaml.dump({'include-in-header':[{'text':r'\usepackage{fvextra}\DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines,breakanywhere,commandchars=\\\{\}}'}],'include-before-body':[{'text':r'\DefineVerbatimEnvironment{verbatim}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines}'}]},f)
      !quarto render {p}/{n.stem}.ipynb --metadata-file={p}/config.yml --to pdf -M latex-auto-install -M margin-top=1in -M margin-bottom=1in -M margin-left=1in -M margin-right=1in --quiet
      google.colab.files.download(str(p/f'{n.stem}.pdf'));s.value=f'✅ Downloaded: {n.stem}.pdf'
    except Exception as e:s.value=f'❌ {str(e)}'
    finally:b.disabled=False
  if not pathlib.Path('/usr/local/bin/quarto').exists():
    !wget -q 'https://quarto.org/download/latest/quarto-linux-amd64.deb' && dpkg -i quarto-linux-amd64.deb>/dev/null && quarto install tinytex --update-path --quiet && rm quarto-linux-amd64.deb
  b=ipywidgets.widgets.Button(description='⬇️ Download');s=ipywidgets.widgets.Label();b.on_click(lambda b:convert(b));IPython.display.display(ipywidgets.widgets.HBox([b,s]))
colab2pdf() # | Colab2PDF v1.6 | https://github.com/drengskapur/colab2pdf | GPL-3.0-or-later |